By Ladji Idrissa Fofana & Zineb Bouharra

### Implications in future researches

#### Non- Negative Matrix Factorization

**Theorical Motivations**

Matrix factorization is a technique used in collaborative filtering to discover latent features that explain the observed ratings. These latent features, such as action level or family friendliness, can be continuous rather than discrete, allowing the model to capture nuances in individual preferences. This is particularly useful for sparse data, such as movie reviews, where not every user has rated the same movies. Additionally, matrix factorization, specifically Singular Value Decomposition (SVD), can be performed in parallel, making it a good choice for large datasets. In the next steps, we will implement a nonnegative matrix factorization model to improve our recommendation engine.

The goal is to approximate the original ratings matrix by finding two matrices, $P$ and $Q$, such that the dot product of $P$ and $Q^T$ is close to the original matrix.

Let $R$ be a sparse $UxM$ matrix of user-movie ratings, where blank spots represent missing ratings. By factoring $R$ into $P$ and $Q$, with $P$ being of size $UxK$ and $Q$ being of size $MxK$, where $K$ is the number of latent features, we can fill in the missing ratings with predicted values. The intuition is that there are underlying features that influence how users rate items, and by discovering these features we can make better predictions.

This task is mathematically represented as: given a set of $U$ users and $M$ movies, find matrices $P$ and $Q$ such that $P$ is $UxK$ and $Q$ is $MxK$, and $PQ^T$ approximates $R$. $P$ represents the strength of the associations between users and latent features, and $Q$ represents the associations of movies with latent features.

$$R \approx P \times Q^T=\hat{R}$$

The model is generally optimized by $$\min _{P, Q}\|R-P Q\|_F \quad$$ such that $\quad P, Q \geq 0$


We used gradient descent to perform matrix factorization by initializing two matrices, $P$ and $Q$, and iteratively adjusting them to minimize the difference between their dot product and the original matrix. This allows the algorithm to find a solution that approximates the original matrix within a certain level of error.



Our method for determining the error used in the factorization process is to calculate the square of the difference between the predicted value and the actual value :

$$e_{i j}=\left(r_{i j}-\hat{r}_{i j}\right)^2$$

In order to minimize the error, we adjust the values of pik and qkj by following the slope of the error function, which is calculated by taking the derivative of the error equation with respect to p :

$$\frac{\partial}{\partial p_{i k}} e_{i j}=-2\left(r_{i j}-\hat{r}{i j}\right)\left(q_{k j}\right)=-2 e_{i j} q_{k j}$$

The error was minimized by adjusting the values of the matrices $p_{k j}$ and $q_{k j}$ by using gradient descent. The gradient of the error equation was calculated by differentiating it with respect to the variables $p$ and $q$, resulting in equations that specify how to adjust the values of $p_{k j}$ and $q_{k j}$ :

$$\frac{\partial}{\partial q_{i k}} e_{i j}=-2\left(r_{i j}-\hat{r}{i j}\right)\left(p_{i k}\right)=-2 e_{i j} p_{i k}$$

The values of matrices $P$ and $Q$ are updated according to the learning rule, which is determined by a constant learning rate $\alpha$. This rate should be chosen carefully, as a too large value may cause the algorithm to step over the minimum, while a too small value may result in slow convergence :

$$p_{i k}^{\prime}=p_{i k}+\alpha \frac{\partial}{\partial p_{i k}} e_{i j}=p_{i k}+2 \alpha e_{i j} q_{k j}$$
\
$$q_{k j}^{\prime}=q_{k j}+\alpha \frac{\partial}{\partial q_{k j}} e_{i j}=q_{k j}+2 \alpha e_{i j} p_{i k}$$

We can stop the iterative process of updating the matrices P and Q when the sum of the error squared falls below a certain threshold value or after a certain number of iterations is reached.

#### Implementation

 The purpose of matrix factorization is to decompose a given matrix $R$ into two matrices $P$ and $Q$, where $P$ represents the user-feature matrix and $Q$ represents the item-feature matrix.After using the __gradient descent algorithm__, the product of these two matrices approximates the original matrix $R$.

In [1]:
import numpy as np

def initialize(R, K):
    """
    Returns initial matrices for an N X M matrix, R and K features.

     R: the matrix to be factorized
     K: the number of latent features

    :returns: P, Q initial matrices of N x K and M x K sizes
    """
    N, M = R.shape # Get the shape of the input matrix R
    P = np.random.rand(N,K) # Initialize the matrix P with random values between 0 and 1 of size N x K
    Q = np.random.rand(M,K) # Initialize the matrix Q with random values between 0 and 1 of size M x K

    return P, Q


In [2]:
def NMF(R, P=None, Q=None, K=2, steps=5000, alpha=0.0002, beta=0.02):
    """
    Performs matrix factorization on R with given parameters.

     R: A matrix to be factorized, dimension N x M
     P: an initial matrix of dimension N x K
     Q: an initial matrix of dimension M x K
     K: the number of latent features
     steps: the maximum number of iterations to optimize in
     alpha: the learning rate for gradient descent
     beta:  the regularization parameter

    :returns: final matrices P and Q
    """

    if not P or not Q:
        P, Q = initialize(R, K) #initialize P and Q if they are not provided
    Q = Q.T # transpose Q

    rows, cols = R.shape
    for step in range(steps): # loop through the maximum number of iterations to optimize

        for i in range(rows):
            for j in range(cols):
                if R[i,j] > 0:
                    eij = R[i,j] - np.dot(P[i,:], Q[:,j]) # calculate the error for R[i,j]
                    for k in range(K):
                        P[i,k] = P[i,k] + alpha * (2 * eij * Q[k,j] - beta * P[i,k]) # update P[i,k]
                        Q[k,j] = Q[k,j] + alpha * (2 * eij * P[i,k] - beta * Q[k,j]) # update Q[k,j]

        e  = 0
        for i in range(rows): # loop through the rows of the matrix R
            for j in range(cols): # loop through the columns of the matrix R
                if R[i,j] > 0:
                    e = e + pow(R[i,j] - np.dot(P[i,:], Q[:,j]), 2) # calculate the total error
                    for k in range(K):
                        e = e + (beta/2) * (pow(P[i,k], 2) + pow(Q[k,j], 2)) # regularization term
        if e < 0.001:
            break

    return P, Q.T # return the final matrices P and Q


#### Test

In [3]:
# Test data
R = np.array([[1,2,0,0], [1,0,0,0], [2,2,0,0], [1,0,0,1], [2,1,0,0]])

# Call the factor function
P, Q = NMF(R, K=5, steps=5000, alpha=0.0002, beta=0.02)

# Print the results
print("Matrix P:")
print(P)
print("Matrix Q:")
print(Q)


Matrix P:
[[ 0.26263117  0.97159489  0.83697281  0.63844373  0.61396705]
 [ 0.30412048  0.66432076  0.23501567  0.14256753  0.81771627]
 [ 1.07023729  0.66152944  0.53657825  0.45153353  0.74706731]
 [ 0.3499086   0.43245018  0.05014987  0.62063163  0.53055552]
 [ 0.94392242  0.13486212 -0.02149571  0.28769792  0.86313328]]
Matrix Q:
[[ 1.21815326 -0.02950767  0.0745381   0.36455737  0.71446784]
 [ 0.52777883  0.51331496  0.78883184  0.56871233  0.47260434]
 [ 0.31828754  0.33330419  0.00748847  0.02085077  0.34487079]
 [ 0.42513636  0.07962145  0.26457821  0.56286521  0.79233743]]


#### NMF- ALSWR - Projected Gradient Descent

In [4]:
import numpy as np

def nmf_als_wr(R, k, l1, l2, tol, max_iter):
    n, m = R.shape  # get the dimensions of the input matrix R
    M = np.random.rand(n, k)  # initialize the matrix M with random values
    U = np.random.rand(k, m)  # initialize the matrix U with random values
    
    for i in range(max_iter):
        # solve for U using least squares, with a regularization term added
        U = np.linalg.solve(M.T @ M + l2 * np.eye(k), M.T @ R)
        U = np.maximum(U, 0)  # project U back into the non-negative space
        
        # solve for M using least squares, with a regularization term added
        M = np.linalg.solve(U @ U.T + l2 * np.eye(k), R @ U.T)
        M = np.maximum(M, 0)  # project M back into the non-negative space
        
        # compute the L2-norm weight regularization term
        M_proj = M / np.sqrt(np.sum(M**2, axis=0))
        M_proj = M_proj * np.minimum(1, np.sqrt(l1 / np.sum(M_proj**2, axis=0)))
        M = M_proj
        
        # compute the reconstruction error
        error = np.linalg.norm(R - M @ U)
        if error < tol:  # check if the error is below the tolerance
            break
    
    return M, U


#### Test

In [5]:
import numpy as np

# Define the input matrix R
R = np.array([[1, 2, 3], [4, 5, 6]])

# Define the number of components k
k = 2

# Define the regularization parameters l1 and l2
l1 = 0.1
l2 = 0.2

# Define the tolerance tol
tol = 1e-6

# Define the maximum number of iterations
max_iter = 100

# Call the nmf_als_wr function
M, U = nmf_als_wr(R, k, l1, l2, tol, max_iter)

# Print the results
print("M:")
print(M)
print("U:")
print(U)


M:
[[0.         0.        ]
 [0.31622777 0.31622777]]
U:
[[3.16227766 3.95284708 4.74341649]
 [3.16227766 3.95284708 4.74341649]]


#### An Alternative NMF without Gradient Descent

The input to the algorithm is a non-negative matrix $R$, the number of components (or rank) of the matrices being decomposed, and optional parameters for the maximum number of iterations and the tolerance for approximation error.

The algorithm initializes two matrices $P$ and $Q$ with random non-negative values, and then iteratively updates them using the multiplicative update rule. Specifically, in each iteration:

The matrix $Q$ is updated by element-wise multiplication with the term $(P^T R) / (P^T P Q)$
The matrix $Q$ is then thresholded to ensure that all its elements are non-negative
The matrix $P$ is updated by element-wise multiplication with the term $(R Q^T) / (P Q Q^T)$
The matrix $P$ is then thresholded to ensure that all its elements are non-negative
The approximation error is computed as the Frobenius norm of the difference between the original matrix $R$ and the product of the two matrices $P$ and $Q$. If the error is below the given tolerance, the algorithm terminates. Otherwise, the algorithm iterates until the maximum number of iterations is reached.

The output of the algorithm is the two matrices $P$ and $Q$ that approximate the original matrix $R$. NMF can be used for various applications such as data compression, feature extraction, and topic modeling.


$$ R ≈ P×Q $$

Where P and Q are the non-negative matrices.






#### NMF Using Sckit- Learn

In [6]:
from sklearn.decomposition import NMF
import numpy as np

# Generate a simulated matrix
np.random.seed(0)
matrix = np.random.randint(10, size=(5, 5))

# Initialize the NMF model
nmf_model = NMF(n_components=2)

# Fit the model to the matrix
nmf_model.fit(matrix)

# Get the factorization matrices
W = nmf_model.transform(matrix)
H = nmf_model.components_

# Reconstruction of the matrix
reconstructed_matrix = np.dot(W, H)

print(reconstructed_matrix - matrix) 

[[ 1.45619443  0.53202653  0.19429266 -1.3950989  -1.45446554]
 [-1.58134923 -0.64240736 -0.15577295  1.53037729  1.54955308]
 [-0.82842747  1.06464528 -0.53986046 -0.13514694  1.21241493]
 [-0.54048483  0.20000553  0.23826174 -0.12478481  0.50938129]
 [ 1.57977729 -0.96161356  0.29399509 -0.12588792 -1.87985208]]


### Practice Using Netflix Competition Dataset

In [7]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("project").getOrCreate()

In [8]:
# Import text data into an RDD
small_ratings_raw_data = spark.sparkContext.textFile("ratings.csv")
# Identify and display the first line
small_ratings_raw_data_header = small_ratings_raw_data.take(1)[0]
print(small_ratings_raw_data_header)
# Create RDD without header
all_lines = small_ratings_raw_data.filter(lambda l : l!=small_ratings_raw_data_header)

userId,movieId,rating,timestamp


In [9]:
#Split the fields (user, item, rating) into a new RDD.
from pyspark.sql import Row
split_lines = all_lines.map(lambda l : l.split(","))
ratingsRDD = split_lines.map(lambda p: Row(user=int(p[0]), item=int(p[1]),
                                     rating=float(p[2]), timestamp=int(p[3])))

# .cache(): the RDD is kept in memory once processed
ratingsRDD.cache()

# Display the two first rows
ratingsRDD.take(2)

[Row(user=1, item=1, rating=4.0, timestamp=964982703),
 Row(user=1, item=3, rating=4.0, timestamp=964981247)]

In [10]:
# Convert RDD to DataFrame
ratingsDF = spark.createDataFrame(ratingsRDD)

### Rank optimization 

The file contains 10,000 ratings cross-referencing the opinions of a thousand users on the movies they have seen among 1700

#### Sampling
Random separation into three samples learning, validation and testing. The rank parameter is optimized by minimizing the error estimate on the test sample. This strategy, rather than cross-validation, is more suited to big data.

In [11]:
tauxTrain=0.6
tauxVal=0.2
tauxTes=0.2
# If the total is less than 1, the data is undersampled.
(trainDF, validDF, testDF) = ratingsDF.randomSplit([tauxTrain, tauxVal, tauxTes])
# Validation and testing to predict
validDF_P = validDF.select("user", "item")
testDF_P = testDF.select("user", "item")

#### NMF rank optimization

The error of imputation of the data, therefore of recommendation, is estimated on the validation sample for different values (grid) of the rank of the matrix factorization.

In principle, the value of the penalty parameter should also be optimised at 0.1 by default.

Important point: the factorization fit error only takes into account the values listed in the hollow matrix, not the "0s" which are missing data.

In [12]:
from pyspark.ml.recommendation import ALS
import math
import collections
# Set the seed
seed = 5
#Number of max iteration for ALS
maxIter = 10
# L1 Regularization; also to be optimized
regularization_parameter = 0.1
# Choice of a grid for optimizing rank values
ranks = [4, 8, 12]
#Initializing variable
# Creating a dictionary to store the error by tested rank
errors = collections.defaultdict(float)
tolerance = 0.02
min_error = float('inf')
best_rank = -1
best_iteration = -1

In [13]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import *
for rank in ranks:
    als = ALS( rank=rank, seed=seed, maxIter=maxIter,
                      regParam=regularization_parameter)
    model = als.fit(trainDF)
    # Validation Sample Forecast
    predDF = model.transform(validDF).select("prediction","rating")
    #Remove unpredicter row due to no-presence of user in the train dataset
    pred_without_naDF = predDF.na.fill(0)
    # Compute the RMSE
    evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
    rmse = evaluator.evaluate(pred_without_naDF)
    print("Root-mean-square error for rank %d = "%rank + str(rmse))
    errors[rank] = rmse
    if rmse < min_error:
        min_error = rmse
        best_rank = rank
# Best result
print('Rang optimal: %s' % best_rank)

Root-mean-square error for rank 4 = 1.1742020059451628
Root-mean-square error for rank 8 = 1.1771770762246312
Root-mean-square error for rank 12 = 1.1776044891236213
Rang optimal: 4


#### Final prediction of the test sample.

In [14]:
# We concatenate here the Train and Validation DataFrames.
trainValidDF = trainDF.union(validDF)
# We create a model with the new completed training Dataframe and the rank set to the optimal value.
als = ALS( rank=best_rank, seed=seed, maxIter=maxIter,
                  regParam=regularization_parameter)
model = als.fit(trainValidDF)
# Predicting on the Test DataFrame
testDF = model.transform(testDF).select("prediction","rating")
#Remove unpredicter row due to no-presence of user in the trai dataset
pred_without_naDF = predDF.na.fill(0)
# Calcul du RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                            predictionCol="prediction")
rmse = evaluator.evaluate(pred_without_naDF)
print("Root-mean-square error for rank %d = "%best_rank + str(rmse))

Root-mean-square error for rank 4 = 0.5352747677834642


In [15]:
# He we Unzip downloaded file which contains our matrix data
import zipfile
zip_ref = zipfile.ZipFile("ml-ratings20M.zip", 'r')
zip_ref.extractall()
zip_ref.close()

In [16]:
# Import the text data into an RDD.
ratings_raw_data = spark.sparkContext.textFile("ratings20M.csv")

# Identify and display the first line
ratings_raw_data_header = ratings_raw_data.take(1)[0]
ratings_raw_data_header

# We now create RDD without header
all_lines = ratings_raw_data.filter(lambda l : l!=ratings_raw_data_header)

In [17]:
# Separate the fields (user, item, rating) into a new RDD
split_lines = all_lines.map(lambda l : l.split(","))
ratingsRDD = split_lines.map(lambda p: Row(user=int(p[0]), item=int(p[1]),
                                     rating=float(p[2]), timestamp=int(p[3])))

# Display the two first rows
ratingsRDD.take(2)

[Row(user=1, item=169, rating=2.5, timestamp=1204927694),
 Row(user=1, item=2471, rating=3.0, timestamp=1204927438)]

In [18]:
# Convert RDD to DataFrame
ratingsDF = spark.createDataFrame(ratingsRDD)

In [19]:
tauxTest=0.1
# If the total is less than 1, the data is undersampled.
(trainTotDF,  testDF) = ratingsDF.randomSplit([1-tauxTest, tauxTest])

In [20]:
# Subsampling learning to test for increasing sizes of this sample.
tauxEch=0.2
(trainDF, DropData) = trainTotDF.randomSplit([tauxEch, 1-tauxEch])

#### Model Estimation

In [21]:
import time
time_start=time.time()
# Set the seed
seed = 5
# Max number of itertion for ALS
maxIter = 10
# L1 penalization
regularization_parameter = 0.1
best_rank = 8
# Estimate for each rank value
als = ALS(rank=rank, seed=seed, maxIter=maxIter,
                      regParam=regularization_parameter)
model = als.fit(trainDF)
time_end=time.time()
time_als=(time_end - time_start)
print("ALS prend %d s" %(time_als))

ALS prend 126 s


#### Prediction Error on the Test sample

In [22]:
# Sample Validation Forecast
predDF = model.transform(testDF).select("prediction","rating")
#Remove unpredicter row due to no-presence of user in the train dataset
pred_without_naDF = predDF.na.fill(0)
# Compute RMSE 
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                            predictionCol="prediction")
rmse = evaluator.evaluate(pred_without_naDF)
print("Root-mean-square error for rank %d = "%best_rank + str(rmse))

Root-mean-square error for rank 8 = 0.5295778529450182


### Generalisation to Tensor Dataset

<img src="Tensor.png" width="350" height="350">

We consider here the dataset as a tensor where the first dimension is users, the second dimension is movies, and the third dimension is ratings. In general, any multi-dimensional array can be considered a tensor, and **NTF** can be applied to decompose it into a sum of rank-1 tensors.

Non-negative tensor factorization (NTF) is a method that decomposes a non-negative tensor (a multi-dimensional array with non-negative entries) into a sum of rank-1 non-negative tensors. It can be used for various tasks such as image and video analysis, data mining, and recommendation systems. In the context of a movie recommendation system, NTF can be used to factorize a user-movie-rating tensor into a user-feature matrix, a movie-feature matrix, and a feature-rating matrix. These matrices can then be used to predict missing ratings, recommend movies to users, and provide insights into the characteristics of users and movies.